## Training VAE

In [1]:
import sys
sys.path.append(r"D:\Study\Intelligence And Learning\Fluid Mechanics\Project 2\Code\NNAero")
from nnaero.models.vae import *
from nnaero.models.training.vae_trainer import *

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

INFO:numexpr.utils:NumExpr defaulting to 12 threads.


## Loading Datasets

In [7]:
from torch.utils.data import DataLoader, TensorDataset

In [18]:
airfoil_df = pd.read_csv(r"C:\Users\mohit\Downloads\airfoil\airfoil.csv")
airfoil_df.head(2)

,Unnamed: 0.1,Unnamed: 0,x,y,le_radius,le_angle,te_thickness,te_angle,max_curvature,max_dydx,smoothness,max_angle_change
0,0,air05_000001.dat,"[1.0, 0.9998494, 0.9993977, 0.9986452, 0.99759...","[0.0021193, 0.0021696, 0.0023205, 0.0025717, 0...",0.009239,169.346767,0.004239,4.066156,165.621277,13.992696,18.662465,12.571719
1,1,air05_000002.dat,"[1.0, 0.9998494, 0.9993977, 0.9986452, 0.99759...","[0.0019936, 0.0020424, 0.0021888, 0.0024327, 0...",0.009180,169.543298,0.003987,3.301608,131.508424,12.289509,17.116958,11.244668


In [24]:
def to_list(s):
    return np.fromstring(s).tolist()

airfoil_df["x"] = airfoil_df["x"].apply(eval)
airfoil_df["y"] = airfoil_df["y"].apply(eval)

x_common = np.array(airfoil_df["x"].iloc[0])
Y = np.array(airfoil_df["y"].tolist())

In [28]:

FEAT_SIZE = 257  # As per your code
model = VAE(feature_size=FEAT_SIZE, latent_size=32)

BATCH_SIZE = 64

dataset = TensorDataset(torch.tensor(Y, dtype=torch.float32))
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)











# 2. Define Configuration
# config = VAETrainingConfig(
#     lr=5e-4, 
    # batch_size=64, 
    # epochs=100,
    # kl_annealing=True,
    # kl_warmup_epochs=20
# )

# 3. Create Trainer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
trainer = VAETrainer(model,
                     optimizer=optimizer,
                    kl_annealing=True,
                    warmup_epochs=20,
                    checkpoint_dir="vae_checkpoints"
                     )

In [35]:
EPOCHS = 5
PATIENCE = 10          # epochs to wait for improvement
MIN_DELTA = 1e-6       # minimum change to qualify as improvement
CHECKPOINT_PATH = "vae_checkpoints/epoch_0_model.pth"

history = []
best_loss = float("inf")
epochs_no_improve = 0

for epoch in range(EPOCHS):
    metrics = trainer.train_epoch(train_loader, epoch)
    loss = metrics["loss"]
    history.append(metrics)

    print(f"Epoch {epoch} | " + " | ".join([f"{k}: {v:.4f}" for k, v in metrics.items()]))

    print(loss, best_loss, MIN_DELTA)
    # --- Check for improvement ---
    if loss < best_loss - MIN_DELTA:
        best_loss = loss
        epochs_no_improve = 0

        # Save checkpoint
        CHECKPOINT_PATH = trainer.save_checkpoint(epoch)

    else:
        epochs_no_improve += 1
        print(f"  ⏸ No improvement for {epochs_no_improve}/{PATIENCE} epochs")

    # --- Early stopping ---
    if epochs_no_improve >= PATIENCE:
        print("🛑 Early stopping triggered")
        break

# --- Load best model ---
checkpoint = torch.load(CHECKPOINT_PATH)
trainer.model.load_state_dict(checkpoint["model_state_dict"])
trainer.optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

print(f"✔ Restored best model from epoch {checkpoint['epoch']}")

INFO:VAETrainer:Checkpoint saved to vae_checkpoints\epoch_0_model.pth


Epoch 0 | loss: 0.0409 | recon_loss: 0.0409 | kl_loss: 0.0004 | beta: 0.0500
0.04094458628192689 inf 1e-06


INFO:VAETrainer:Checkpoint saved to vae_checkpoints\epoch_1_model.pth


Epoch 1 | loss: 0.0409 | recon_loss: 0.0409 | kl_loss: 0.0002 | beta: 0.1000
0.040885700730299485 0.04094458628192689 1e-06


INFO:VAETrainer:Checkpoint saved to vae_checkpoints\epoch_2_model.pth


...

INFO:VAETrainer:Checkpoint saved to vae_checkpoints\epoch_3_model.pth


Epoch 3 | loss: 0.0408 | recon_loss: 0.0408 | kl_loss: 0.0001 | beta: 0.2000
0.04079207555906286 0.04081108128489221 1e-06


INFO:VAETrainer:Checkpoint saved to vae_checkpoints\epoch_4_model.pth


Epoch 4 | loss: 0.0408 | recon_loss: 0.0408 | kl_loss: 0.0001 | beta: 0.2500
0.04076991698623282 0.04079207555906286 1e-06


AttributeError: 'NoneType' object has no attribute 'seek'. You can only torch.load from a file that is seekable. Please pre-load the data into a buffer like io.BytesIO and try to load from it instead.